In [1]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model
from qiskit import Aer
from qiskit.aqua import aqua_globals, QuantumInstance


backend = BasicAer.get_backend('statevector_simulator')

In [2]:
model = Model()
x0 = model.binary_var(name='x0')
x1 = model.binary_var(name='x1')
x2 = model.binary_var(name='x2')
model.minimize(-x0+2*x1-3*x2-2*x0*x2-1*x1*x2)
qp = QuadraticProgram()
qp.from_docplex(model)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - x0 + 2 x1 - 3 x2 + [ - 4 x0*x2 - 2 x1*x2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End



In [3]:
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)
results = grover_optimizer.solve(qp)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

x=[1 0 1]
fval=-6.0


In [7]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print("x={}".format(exact_result.x))
print("fval={}".format(exact_result.fval))

x=[1.0, 0.0, 1.0]
fval=-6.0


In [12]:
q_instance = QuantumInstance(Aer.get_backend('statevector_simulator'),
                                          seed_simulator=921, seed_transpiler=200)

In [13]:
# When the answer is zero

# Input.
model = Model()
x_0 = model.binary_var(name='x0')
x_1 = model.binary_var(name='x1')
model.minimize(0*x_0+0*x_1)
op = QuadraticProgram()
op.from_docplex(model)

# Will not find a negative, should return 0.
gmf = GroverOptimizer(1, num_iterations=1, quantum_instance=q_instance)
results = gmf.solve(op)
print(results.x)
print(results.fval)

[0.0, 0.0]
0


In [18]:
def validate_results(problem, results):
        """Validate the results object returned by GroverOptimizer."""
        # Get expected value.
        solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
        comp_result = solver.solve(problem)

        # Validate results.
        print(comp_result.x == results.x)
        print(comp_result.fval == results.fval)

In [19]:
# A simple case, with 2 linear coeffs and no quadratic coeffs or constants

# Input.
model = Model()
x_0 = model.binary_var(name='x0')
x_1 = model.binary_var(name='x1')
model.minimize(-x_0+2*x_1)
op = QuadraticProgram()
op.from_docplex(model)

# Get the optimum key and value.
n_iter = 8
gmf = GroverOptimizer(4, num_iterations=n_iter, quantum_instance=q_instance)
results = gmf.solve(op)
validate_results(op, results)

True
True


In [20]:
# Simple case, but with maximization

# Input.
model = Model()
x_0 = model.binary_var(name='x0')
x_1 = model.binary_var(name='x1')
model.maximize(-x_0+2*x_1)
op = QuadraticProgram()
op.from_docplex(model)

# Get the optimum key and value.
n_iter = 8
gmf = GroverOptimizer(4, num_iterations=n_iter, quantum_instance=q_instance)
results = gmf.solve(op)
validate_results(op, results)

False
False


In [21]:
# Example from https://arxiv.org/abs/1912.04088."""

# Input.
model = Model()
x_0 = model.binary_var(name='x0')
x_1 = model.binary_var(name='x1')
x_2 = model.binary_var(name='x2')
model.minimize(-x_0+2*x_1-3*x_2-2*x_0*x_2-1*x_1*x_2)
op = QuadraticProgram()
op.from_docplex(model)

# Get the optimum key and value.
n_iter = 10
gmf = GroverOptimizer(6, num_iterations=n_iter, quantum_instance=q_instance)
results = gmf.solve(op)
validate_results(op, results)

True
True
